In [ ]:
import pickle
from pathlib import Path
from collections import Counter
import pandas as pd
import hashed_index
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [45]:
import importlib
importlib.reload(hashed_index)

<module 'hashed_index' from '/content/hashed_index.py'>

In [ ]:
# Paths
# Using user page views (as opposed to spiders and automated traffic) for the 
# month of August 2021
pv_path = 'https://dumps.wikimedia.org/other/pageview_complete/monthly/2021/2021-08/pageviews-202108-user.bz2'
p = Path(pv_path) 
pv_name = p.name
pv_temp = f'{p.stem}-4dedup.txt'
pv_clean = f'{p.stem}.pkl'
# Download the file (2.3GB) 
print(pv_name)
print(pv_temp)
print(pv_clean)


!wget -N $pv_path
# Filter for English pages, and keep just two fields: article ID (3) and monthly 
# total number of page views (5). Then, remove lines with article id or page 
# view values that are not a sequence of digits.
!bzcat $pv_name | grep "^en\.wikipedia" | cut -d' ' -f3,5 | grep -P "^\d+\s\d+$" > $pv_temp
# Create a Counter (dictionary) that sums up the pages views for the same 
# article, resulting in a mapping from article id to total page views.



pageviews-202108-user.bz2
pageviews-202108-user-4dedup.txt
pageviews-202108-user.pkl
--2022-01-06 11:56:27--  https://dumps.wikimedia.org/other/pageview_complete/monthly/2021/2021-08/pageviews-202108-user.bz2
Resolving dumps.wikimedia.org (dumps.wikimedia.org)... 208.80.154.7, 2620:0:861:1:208:80:154:7
Connecting to dumps.wikimedia.org (dumps.wikimedia.org)|208.80.154.7|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2503235912 (2.3G) [application/octet-stream]
Saving to: ‘pageviews-202108-user.bz2’

pageviews-202108-us 100%[===================>]   2.33G  4.35MB/s    in 9m 27s  

2022-01-06 12:05:54 (4.21 MB/s) - ‘pageviews-202108-user.bz2’ saved [2503235912/2503235912]



In [46]:
wid2pv = Counter()
with open(pv_temp, 'rt') as f:
  for line in f:
    parts = line.split(' ')
    wid2pv.update({int(parts[0]): int(parts[1])})

print(len(list(wid2pv.items())))
print(list(wid2pv.items())[:100])

10771932
[(5878274, 52), (7712754, 28713), (3632887, 528), (600744, 4285), (59804426, 61650), (63743203, 23), (34443176, 7), (4838455, 1450), (11011780, 9), (4822278, 779), (34443184, 4), (2556962, 594), (59827823, 1), (59827819, 2), (39401265, 1), (10065458, 6), (55029148, 123), (61082890, 1), (16381751, 209), (55029832, 4), (16789001, 2), (4700151, 1322), (53802835, 2), (5840274, 165), (1669534, 6340), (5835176, 3), (36372278, 2), (714570, 429), (35277272, 1), (712986, 1), (594417, 7342), (63079087, 3), (1202168, 8311), (2122994, 1), (321544, 18535), (918987, 1), (6449075, 1), (16645718, 4), (8355009, 64), (54365640, 4), (1669616, 4), (10936290, 27), (16491695, 4), (2978668, 288), (16395685, 2), (2349724, 4248), (16563299, 1), (4906617, 1997), (16389493, 2), (8004648, 93), (143804, 273), (1222755, 908), (16245639, 3), (16237771, 1), (4077845, 73), (16583252, 3), (5297162, 333), (1163095, 408), (14876253, 69), (16222834, 1), (9378013, 79), (16500046, 2), (6395043, 254), (1729032, 5426

In [47]:
sort_dic = sorted(list(wid2pv.items()),key=lambda x: x[0])
print(sort_dic[:20])
print(sort_dic[-20:])
hashed_index.write_a_dictionary(sort_dic, 'drive/MyDrive/Test Data/pv/', 'pv.pkl' )

[(10, 110), (12, 75195), (13, 6), (14, 3), (15, 1), (19, 1), (20, 2), (21, 1), (23, 1), (25, 114746), (27, 7), (29, 1), (30, 1), (36, 1), (39, 24532), (40, 1), (42, 3), (46, 3), (50, 2), (52, 1)]
[(68616009, 1), (68616010, 1), (68616011, 2), (68616012, 4), (68616014, 1), (68616015, 2), (68616016, 1), (68616017, 20), (68616019, 1), (68616020, 1), (68616021, 1), (68616022, 3), (68616023, 5), (68616024, 1), (68616025, 1), (68616027, 3), (68616030, 1), (68616031, 1), (68616033, 2), (68616034, 1)]


In [48]:
import time
t = time.time()
print(hashed_index.get_value('drive/MyDrive/Test Data/pv/', 'pv.pkl', 12))
print(hashed_index.get_value('drive/MyDrive/Test Data/pv/', 'pv.pkl', 68616034))
print(hashed_index.get_value('drive/MyDrive/Test Data/pv/', 'pv.pkl', 39))
print(time.time() - t)

75195
1
24532
0.13550472259521484
